In [1]:
import mujoco
import mujoco.viewer
import numpy as np
import time
# 加载宇树机器人模型
model = mujoco.MjModel.from_xml_path(r"unitree_rl_gym\resources\robots\g1_description\g1_29dof.xml")
data = mujoco.MjData(model)

In [4]:
# 创建查看器
with mujoco.viewer.launch_passive(model, data) as viewer:
    # 设置相机视角
    viewer.cam.distance = 5
    viewer.cam.azimuth = 140
    viewer.cam.elevation = -20
    
    # 模拟循环
    for _ in range(10000):
        # 生成随机控制信号（在关节限制范围内）
        for i in range(model.nu):
            ctrl_range = model.actuator_ctrlrange[i]
            print("ctrl_range:",ctrl_range)
            # if not np.all(np.isinf(ctrl_range)):  # 如果有关节限制
            #     data.ctrl[i] = np.random.uniform(ctrl_range[0], ctrl_range[1])
            # else:
            # data.ctrl[i] = np.random.uniform(-10, 10)  # 对于无限制关节
            data.ctrl[4] = -1000
            print(f"data.ctrl[{i}]: ",data.ctrl[i])
            print("np.all(np.isinf(ctrl_range)):",np.all(np.isinf(ctrl_range)))
        
        # 步进模拟
        mujoco.mj_step(model, data)
        
        # 更新查看器
        viewer.sync()
        
        # 添加小延迟以便观察
        time.sleep(0.01)

ctrl_range: [0. 0.]
data.ctrl[0]:  0.0
np.all(np.isinf(ctrl_range)): False
ctrl_range: [0. 0.]
data.ctrl[1]:  0.0
np.all(np.isinf(ctrl_range)): False
ctrl_range: [0. 0.]
data.ctrl[2]:  0.0
np.all(np.isinf(ctrl_range)): False
ctrl_range: [0. 0.]
data.ctrl[3]:  0.0
np.all(np.isinf(ctrl_range)): False
ctrl_range: [0. 0.]
data.ctrl[4]:  -1000.0
np.all(np.isinf(ctrl_range)): False
ctrl_range: [0. 0.]
data.ctrl[5]:  0.0
np.all(np.isinf(ctrl_range)): False
ctrl_range: [0. 0.]
data.ctrl[6]:  0.0
np.all(np.isinf(ctrl_range)): False
ctrl_range: [0. 0.]
data.ctrl[7]:  0.0
np.all(np.isinf(ctrl_range)): False
ctrl_range: [0. 0.]
data.ctrl[8]:  0.0
np.all(np.isinf(ctrl_range)): False
ctrl_range: [0. 0.]
data.ctrl[9]:  0.0
np.all(np.isinf(ctrl_range)): False
ctrl_range: [0. 0.]
data.ctrl[10]:  0.0
np.all(np.isinf(ctrl_range)): False
ctrl_range: [0. 0.]
data.ctrl[11]:  0.0
np.all(np.isinf(ctrl_range)): False
ctrl_range: [0. 0.]
data.ctrl[12]:  0.0
np.all(np.isinf(ctrl_range)): False
ctrl_range: [0. 0.

KeyboardInterrupt: 

In [12]:
data.ctrl

array([    0.,     0.,     0.,     0., -1000.,     0.,     0.,     0.,
           0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
           0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
           0.,     0.,     0.,     0.,     0.])

In [3]:
data.qpos.copy()

array([0.   , 0.   , 0.793, 1.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
       0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
       0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
       0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ])

In [4]:
# 更新运动学信息
mujoco.mj_kinematics(model, data)

# 获取所有刚体的笛卡尔坐标
body_positions = data.xpos

# 打印所有刚体的笛卡尔坐标
for i in range(model.nbody):
    body_name = mujoco.mj_id2name(model, mujoco.mjtObj.mjOBJ_BODY, i)
    print(f"Body {i}: {body_name}, Position: {body_positions[i]}")

Body 0: world, Position: [0. 0. 0.]
Body 1: pelvis, Position: [0.    0.    0.793]
Body 2: left_hip_pitch_link, Position: [0.       0.064452 0.6903  ]
Body 3: left_hip_roll_link, Position: [0.       0.116452 0.659835]
Body 4: left_hip_yaw_link, Position: [0.04621766 0.116452   0.541959  ]
Body 5: left_knee_link, Position: [-2.33090489e-06  1.18600900e-01  3.53704248e-01]
Body 6: left_ankle_pitch_link, Position: [-2.33090489e-06  1.18506455e-01  5.36942476e-02]
Body 7: left_ankle_roll_link, Position: [-2.33090489e-06  1.18506455e-01  3.61362476e-02]
Body 8: right_hip_pitch_link, Position: [ 0.       -0.064452  0.6903  ]
Body 9: right_hip_roll_link, Position: [ 0.       -0.116452  0.659835]
Body 10: right_hip_yaw_link, Position: [ 0.04621766 -0.116452    0.541959  ]
Body 11: right_knee_link, Position: [-2.33090489e-06 -1.18600900e-01  3.53704248e-01]
Body 12: right_ankle_pitch_link, Position: [-2.33090489e-06 -1.18506455e-01  5.36942476e-02]
Body 13: right_ankle_roll_link, Position: [-2.3

In [5]:
import mujoco
import numpy as np
import time

# 创建查看器
with mujoco.viewer.launch_passive(model, data) as viewer:
    # 设置相机视角
    viewer.cam.distance = 5
    viewer.cam.azimuth = 140
    viewer.cam.elevation = -20

    # 定义目标位置
    target_body_id = 2  # body30 的 ID
    target_pos = np.array([0.3, -0.2, 0.8])  # 目标位置

    # 操作空间控制参数
    kp = 100.0  # 位置增益
    kd = 20.0   # 速度增益

    # 模拟循环
    for _ in range(1000):
        # 更新运动学信息
        mujoco.mj_kinematics(model, data)

        # 计算雅可比矩阵
        J_pos = np.zeros((3, model.nv))
        J_rot = np.zeros((3, model.nv))
        mujoco.mj_jacBody(model, data, J_pos, J_rot, target_body_id)

        # 计算位置误差
        pos_error = target_pos - data.xpos[target_body_id]

        # 将位置误差扩展到与 data.qvel 相同的维度
        error = np.zeros(model.nv)
        error[:3] = pos_error

        # 计算关节扭矩
        joint_torques = kp * error - kd * data.qvel

        # 应用关节扭矩
        data.qfrc_applied[:] = joint_torques

        # 步进模拟
        mujoco.mj_step(model, data)

        # 更新查看器
        viewer.sync()

        # 添加小延迟以便观察
        time.sleep(0.01)

        # 如果达到目标位置，退出循环
        if np.linalg.norm(data.xpos[target_body_id] - target_pos) < 0.01:
            break

In [25]:
import mujoco
import numpy as np
from mujoco.viewer import launch

# 加载模型
model = mujoco.MjModel.from_xml_path(r"unitree_rl_gym\resources\robots\g1_description\g1_29dof.xml")
data = mujoco.MjData(model)

# 设置关节控制
def on_step(viewer, model, data):
    # 随机设置关节控制值
    data.ctrl[:] = np.random.uniform(-1, 1, size=model.nu)

# 启动可视化
launch(model=model, data=data, on_step=on_step)

TypeError: launch() got an unexpected keyword argument 'on_step'